In [1]:
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
import time
import csv
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests

In [2]:
def amz_comments(product):
    """This function will take amazon product name as input and then extract all the comments 
       and respective ratings, save them in a csv file. Capped no of comments per rating at 100 per product,
       to avoid imbalance (Products have 1,4,5 ratings comments in abundance, while 2 and 3 ratings are
       low in number). Selenium and BeautifulSoup will be used for scraping"""
    
    # Setup selenium webdriver, opening new window
    driver = webdriver.Firefox(executable_path = GeckoDriverManager().install())
    driver.execute_script("window.open('');")
    
    # Setup csvfile writer
    csv_file = open('{}.csv'.format(product), 'w')
    writer = csv.writer(csv_file)
    writer.writerow(['Comment', 'Rating'])
    
    # Opening product page on amazon.in
    driver.get('https://www.amazon.in')
    time.sleep(3)                                     #3sec break or else amazon kicks us out.
    driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]').send_keys(product)
    driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]').send_keys(Keys.RETURN)
    time.sleep(3)
    
    #Loop 1: Pagination
    global_comment_count = 0
    condition = True
    page_no = 1
    #The condition for this loop is: either global_comment_count = 5000 or page_no = 10
    while condition:
        print('Product Page {} opened'.format(page_no))
        print('global comment count = ', global_comment_count)
        #Extracting product urls
        html = driver.execute_script("return document.body.innerHTML;")
        soup_page = BeautifulSoup(html)
        urls = soup_page.findAll("a", {"class":"a-link-normal a-text-normal"}) #This list will contain all product url on page
        if len(urls)>0:
            print('HTML Parsed')
        else:
            break
        driver.switch_to.window(driver.window_handles[1])
        #Loop2 looping through individual products:
        for url in urls:
            try:
                driver.get('https://amazon.in'+url['href'])
                time.sleep(3)
            except:
                continue
            #some products have no comments, using try except block to avoid errors
            try:
                html = driver.execute_script("return document.body.innerHTML;")
                soup = BeautifulSoup(html)
                url_comments_page = 'https://amazon.in'+soup.find('a', {'class' : 'a-link-emphasis a-text-bold'})['href']
                driver.get(url_comments_page)
            except:
                print('No comments found, trying next product')
                continue
            
            #Loop 3: looping through ratings: 1,2,3,4,5
            for rating in range(5,0,-1):
                # Ratings selected via drop down menu
                time.sleep(3)
                driver.find_element_by_xpath('//*[@id="a-autoid-6-announce"]').click()
                time.sleep(3)
                driver.find_element_by_xpath('//*[@id="star-count-dropdown_{}"]'.format(rating)).click()
                
                # Now we try to extract 100 comments per rating per product. Pagination is reqd
                # Loop 4: Extracting comments
                com_count = 0
                while (com_count<101):
                    # Using beautifulsoup to extract comments
                    html = driver.execute_script("return document.body.innerHTML;")
                    soup = BeautifulSoup(html)
                    comments = soup.findAll('span', {'class' : 'a-size-base review-text review-text-content'})
                    for comment in comments:
                        com = comment.get_text().strip()
                        writer.writerow([com.encode('utf-8'), rating])
                        com_count = com_count+1
                    try:
                        nxt_comment_page_url = soup.findAll('li', {'class':'a-last'})[0].a['href']
                        driver.get('https://amazon.in'+nxt_comment_page_url)
                    except:
                        break
                    # **Loop 4 End**
                global_comment_count = global_comment_count+com_count
            # **loop 3 End**
            
        # **Loop 2 Ends**
        page_no = page_no+1
        #Pagination for Loop 1
        page_url = 'https://amazon.in'+soup_page.find('li', {'class':'a-last'}).a['href']
        driver.switch_to.window(driver.window_handles[0])
        driver.get(page_url)
        time.sleep(5)
        print('Next Page Opened')
        
        if (page_no > 10) or (global_comment_count>5000):
            condition = False
            print('Total comments extracted = ', global_comment_count)
    csv_file.close()

In [40]:
amz_comments('smartphones')

[WDM] - ====== WebDriver manager ======


[WDM] - Driver [C:\Users\Aditya\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


Product Page 1 opened
global comment count =  0
HTML Parsed
No comments found, trying next product
No comments found, trying next product
Next Page Opened
Total comments extracted =  9157


In [43]:
prod = ['laptops', 'dslr', 'smartwatches', 'earphones', 'speakers', 'trimmers']

for i in prod:
    amz_comments(i)

[WDM] - ====== WebDriver manager ======


[WDM] - Driver [C:\Users\Aditya\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


Product Page 1 opened
global comment count =  0
HTML Parsed


[WDM] - ====== WebDriver manager ======


Next Page Opened
Total comments extracted =  2157




...

[WDM] - Driver [C:\Users\Aditya\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


Product Page 1 opened
global comment count =  0
HTML Parsed


[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\Aditya\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


Next Page Opened
Total comments extracted =  12564


Product Page 1 opened
global comment count =  0
HTML Parsed


ElementClickInterceptedException: Message: Element <span id="a-autoid-6-announce" class="a-button-text a-declarative"> is not clickable at point (418,112) because another element <a id="star-count-dropdown_0" class="a-dropdown-link" href="javascript:void(0)"> obscures it


In [53]:
amz_comments('TV')

[WDM] - ====== WebDriver manager ======


[WDM] - Getting latest mozilla release info for v0.29.1
[WDM] - Trying to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.29.1/geckodriver-v0.29.1-win64.zip
[WDM] - Driver has been saved in cache [C:\Users\Aditya\.wdm\drivers\geckodriver\win64\v0.29.1]


Product Page 1 opened
global comment count =  0
HTML Parsed
Next Page Opened
Total comments extracted =  8365


In [54]:
import pandas as pd
mobiles = pd.read_csv('smartphones.csv')
laptops = pd.read_csv('laptops.csv')
dslr = pd.read_csv('dslr.csv')
watches = pd.read_csv('smartwatches.csv')
earphones = pd.read_csv('earphones.csv')
speakers = pd.read_csv('speakers.csv')
trimmers = pd.read_csv('trimmers.csv')
tv = pd.read_csv('TV.csv')

In [55]:
df = pd.concat([mobiles, laptops, dslr, watches, earphones, speakers, trimmers, tv])

In [56]:
df.head()

,Comment,Rating
0,b'Please be aware that this is a downgraded ve...,5
1,"b""What a Smartphone by Xiaomi! I am thoroughly...",5
2,"b""Received my Mi 11x Pro today\xf0\x9f\x98\xbb...",5
3,"b""I love gaming and have been waiting for a SD...",5
4,"b'One of the best phones from Xiaomi, display ...",5


In [57]:
df.shape

(62580, 2)

In [58]:
df.groupby('Rating').count()

,Comment
Rating,
1,11591
2,11370
3,11189
4,12500
5,15930


In [59]:
df.to_csv('Full_comments_dataset.csv')